# Ensemble (voting)

In [18]:
from src.si.data import Dataset, summary
from si.util import CrossValidationScore
import os

In [19]:
DIR = os.path.dirname(os.path.realpath('.'))
filename = os.path.join(DIR, 'datasets/breast-bin.data')
dataset = Dataset.from_data(filename)
summary(dataset)

,A,B,C,D,E,F,G,H,I,Y
mean,4.417740,3.134478,3.207439,2.806867,3.216023,3.486409,3.437768,2.866953,1.589413,0.344778
var,7.917053,9.298082,8.819630,8.141527,4.896110,13.099601,5.937114,9.311340,2.937284,0.225906
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000


In [20]:
# Use accuracy as scorring function
from si.util import accuracy_score

### Decision Tree

In [21]:
from si.supervised import DecisionTree

In [22]:
dt = DecisionTree()

In [23]:
cv = CrossValidationScore(dt,dataset,score=accuracy_score, flip=True)
cv.run()
cv.toDataframe()

,Train Scores,Test Scores
0,0.957066,0.928571
1,0.946333,0.935714
2,0.957066,0.950000


### Logistic regression

In [24]:
from si.supervised import LogisticRegression
logreg = LogisticRegression()


In [25]:
cv = CrossValidationScore(logreg,dataset,score=accuracy_score, flip=True)
cv.run()
cv.toDataframe()

,Train Scores,Test Scores
0,0.966011,0.978571
1,0.964222,0.964286
2,0.964222,0.964286


### KNN

In [26]:
from si.supervised import Knn

In [27]:
knn = Knn(7)

In [28]:
cv = CrossValidationScore(knn,dataset,score=accuracy_score, flip=True)
cv.run()
cv.toDataframe()

,Train Scores,Test Scores
0,0.976744,0.950000
1,0.976744,0.957143
2,0.980322,0.957143


## Ensemble

In [29]:
def fvote(preds):
    return max(set(preds), key=preds.count)

In [30]:
from si.supervised import Ensemble
en = Ensemble([dt,logreg,knn],fvote,accuracy_score)

In [31]:
cv = CrossValidationScore(en,dataset,score=accuracy_score, flip=True)
cv.run()
cv.toDataframe()

,Train Scores,Test Scores
0,0.969589,0.935714
1,0.973166,0.978571
2,0.967800,0.985714


## Confusion Matrix

In [32]:
from si.supervised import ConfusionMatrix
cm = ConfusionMatrix(cv.true_Y, cv.pred_Y)
cm()
cm.toDataframe()

Predicted,0,1,All
Actual,,,
0.0,270,8,278
1.0,6,136,142
All,276,144,420
